In [4]:
import pandas as pd
import numpy as np
import requests as re
from bs4 import BeautifulSoup

In [261]:
url = 'https://www.indeed.com/r/Data-Scientist-Intern/404950f566637da6'
this_content = re.get(url)

In [262]:
urls_splits =url.split('/')
urls_splits[len(urls_splits)-1]

'404950f566637da6'

In [263]:
soup = BeautifulSoup(this_content.content,'html.parser')

In [264]:
mydict= {}
urls_splits =url.split('/')
mydict['id'] = urls_splits[len(urls_splits)-1]
mydict['summary_title'] = soup.find('h1', attrs={'class': 'fn'}).contents[0]

work_ex= soup.find('div',attrs={'id': 'work-experience-items'})
if len(work_ex.contents)>0:
    mydict['current_job_company']= work_ex.contents[0].find('div', attrs= {'class':'work_company'}).contents[0].contents
    mydict['current_job_title']= work_ex.contents[0].find('p', attrs={'class':'work_title title'}).contents[0]
    mydict['current_job_desc'] = work_ex.contents[0].find('p', attrs={'class':'work_description'}).contents
    mydict['current_job_duration'] = work_ex.contents[0].find('p', attrs={'class':'work_dates'}).contents[0]
    
if len(work_ex.contents)>1:
    # Previous experience is available
    mydict['prev_job_company']= work_ex.contents[1].find('div', attrs= {'class':'work_company'}).contents[0].contents
    mydict['prev_job_title']= work_ex.contents[1].find('p', attrs={'class':'work_title title'}).contents[0]
    mydict['prev_job_desc'] = work_ex.contents[1].find('p', attrs={'class':'work_description'}).contents
    mydict['prev_job_duration'] = work_ex.contents[1].find('p', attrs={'class':'work_dates'}).contents[0]
    
educations = soup.find('div',attrs={'id': 'education-items'})
if len(educations.contents)>0:
    mydict['education1_title']=educations.contents[0].find('p', attrs={'class':'edu_title'}).contents[0]
    mydict['education1_college']=educations.contents[0].find('div', attrs={'class':'edu_school'}).contents[0].contents[0]
    mydict['education1_duration']=educations.contents[0].find('p', attrs={'class':'edu_dates'}).contents[0]

if len(educations.contents)>1:
    mydict['education2_title']=educations.contents[1].find('p', attrs={'class':'edu_title'}).contents[0]
    mydict['education2_college']=educations.contents[1].find('div', attrs={'class':'edu_school'}).contents[0].contents[0]
    mydict['education2_duration']=educations.contents[1].find('p', attrs={'class':'edu_dates'}).contents[0]
    
if len(educations.contents)>2:
    mydict['education3_title']=educations.contents[2].find('p', attrs={'class':'edu_title'}).contents[0]
    mydict['education3_college']=educations.contents[2].find('div', attrs={'class':'edu_school'}).contents[0].contents[0]
    mydict['education3_duration']=educations.contents[2].find('p', attrs={'class':'edu_dates'}).contents[0]
    
    
    
# Skill
skill_list=[]
skills = soup.find('div',attrs={'id': 'skills-items'})
if skills is not None:
    skill_set = skills.contents[0].contents[0].contents
    if len(skill_set)>0:
        for skill in skill_set:
            try:
                skill_list.append(skill.contents)
            except:
                pass
mydict['skils']=skill_list

# links
links_list=[]
links= soup.find('div',attrs={'id':'link-items'})
if links is not None:
    links_set =links.contents
    for link in links_set:
        links_list.append(link.contents[0].contents[0].contents[0].contents[0])

mydict['links']=links_list


# additional_info
add_info = soup.find('div', attrs={'id':'additionalinfo-section'})
if add_info is not None:
    mydict['additional_info']=add_info.contents[0].contents[0].contents


certi_list=[]
certifications = soup.find('div', attrs={'id':'certification-items'})
if certifications is not None:
    for certi in certifications.contents:
        certi_list.append(certi.contents[0].contents[0].contents[0])
mydict['certifications']=certi_list

award_list=[]
awards=soup.find('div', attrs={'id':'award-items'})
if awards is not None:
    for one_award in awards:
        award_list.append(one_award.contents[0].contents[0].contents[0])
        
mydict['awards'] = award_list


In [265]:
mydict

{'additional_info': ['SKILLS\xa0',
  <br/>,
  '•    Data Analytics Tools/Languages: SQL(PostgeSQL, MySQL), NoSQL(MongoDB), Python(Numpy,\xa0',
  <br/>,
  'Pandas, Scipy, Scikit-learn, etc), R, Tableau, Spark, Hive, MapReduce, AWS(S3, EC2, EMR), Github\xa0',
  <br/>,
  '•    Modeling and Statistical Analysis: Regression Analysis, Tree Based Models, kNN, Naive Bayes, Neural\xa0',
  <br/>,
  'Networks, Clustering, PCA, SVD, NMF, Recommender Systems, Hypothesis Testing, Experimental Design'],
 'awards': [],
 'certifications': [],
 'current_job_company': ['Booxby'],
 'current_job_desc': ['•    Booxby is a machine learning platform solving the book discovery problem of publishing industry\xa0',
  <br/>,
  '•    Leading the project of predicting the success of books to help publishers allocate marketing budget\xa0',
  <br/>,
  '•    Using machine learning and natural language processing to help authors and publishers find the right market for their books, and help readers find the right books

In [259]:
pd.DataFrame(data=[mydict])

,awards,certifications,current_job_company,current_job_desc,current_job_duration,current_job_title,education1_college,education1_duration,education1_title,education2_college,education2_duration,education2_title,id,links,prev_job_company,prev_job_desc,prev_job_duration,prev_job_title,skils,summary_title
0,[Stony Brook Turner Graduate Student Fellowshi...,[],[Kickwheel (Formerly Schoold)],[o Engineered pipelines and developed hybri...,September 2017 to Present,Senior Data Scientist,Stony Brook University,2014,PhD in Ecology and Evolution,New York University,2006,BA in Biology,6bfdff58f8024d3c,[http://www.linkedin.com/in/rociong],[Schoold],[o Advised and aided product and engineeri...,September 2015 to September 2017,Data Scientist,"[[Python (3 years)], [R (4 years)], [Spark (Le...",Senior Data Scientist


In [260]:
mydict

{'awards': ['Stony Brook Turner Graduate Student Fellowship',
  'New York University Presidential Honors Scholarship'],
 'certifications': [],
 'current_job_company': ['Kickwheel (Formerly Schoold)'],
 'current_job_desc': ['o    Engineered pipelines and developed hybrid recommendation models (content-based and collaborative filtering) in Spark(ML) for matching student users with relevant content (eg. schools, majors, and careers) based on implicit feedback and profile information \xa0',
  <br/>,
  'o    Advised leadership and cross-functional collaborators on defining and tracking KPIs and metrics and utilizing product insights for driving product and business decisions\xa0',
  <br/>,
  'o    Completed dashboards and product analytics projects, which included data visualizations and communication of findings pertaining to app engagement, retention, growth, user funnels etc. \xa0',
  <br/>,
  'o    Made improvements to Spark ETL pipelines that processed raw analytics data, which led to 

In [109]:
educations.contents[0].find('div', attrs={'class':'edu_school'}).contents[0].contents

['Carnegie Mellon University']

In [144]:
for c in soup.find('div',attrs={'id': 'skills-items'}).contents[0].contents[0].contents:
    try:
        print(c.contents)
    except:
        pass

['Python (4 years)']
['R programming (4 years)']
['SQL (1 year)']


In [139]:
c.contents

['SQL (1 year)']